In [1]:
from __UnpairAIRSATBIG_TFAArea__ import define_discriminator, define_discriminator_content, define_generator, define_generator_content
from __UnpairAIRSATBIG_TFAArea__ import define_composite_model_A2B, define_composite_model_B2A, Encoder, Decoder, train
from __init__ import glob_all
from tensorflow.keras import backend as K
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().disabled = True

import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
tf.config.optimizer.set_jit(True)

In [2]:
channel = 6; resnet_n = 9; d_lr=5e-5; g_lr = 1e-3 ; epochs = 300; batch = 1

AIR73_train = np.load('AIR_DomainA_Training.npy', mmap_mode='r')
SAT73_train = np.load('SAT_DomainA_Training.npy', mmap_mode='r')
dataA = np.concatenate([AIR73_train,SAT73_train])
dataB = np.load('AIRSAT_DomainB_Code'+str(channel)+'_Training.npy', mmap_mode='r')
print('Loaded', dataA.shape, dataB.shape)

EPA73_testing_PM25 = np.load('EPA73_Testing_348X204_PM25.npy', mmap_mode='r')
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
exArea_KNN = np.load('RandomTesting/EPA73_testing_KNN3fill_area.npy', mmap_mode='r')
area_list = ['Taipei_area', 'Zhumiao_area', 'Taichung_area', 'YunChiNan_area', 'KaoPing_area']

model_path = 'TFAArea_CWBEPA_BIGD_Unpair_TE_CycleGan_AIRSAT_RMSprop_Res'+str(resnet_n)+'_Channel'+str(channel)+'_Lr:'+str(g_lr)+'/'
if not os.path.exists(model_path+'result/'):os.makedirs(model_path+'result/')

Loaded (3194, 348, 204, 1) (3194, 44, 26, 6)


In [3]:
Train_Rain = np.concatenate([np.load('../CWB/CWBEPA_Content_RAIN_code_Training_K3.npy'),
                             np.load('../CWB/CWBEPA_Content_RAIN_code_Training_K3.npy')])

Train_Rh = np.concatenate([np.load('../CWB/CWBEPA_Content_RH_code_Training_K3.npy'),
                             np.load('../CWB/CWBEPA_Content_RH_code_Training_K3.npy')])

Train_Temp = np.concatenate([np.load('../CWB/CWBEPA_Content_TEMP_code_Training_K3.npy'),
                             np.load('../CWB/CWBEPA_Content_TEMP_code_Training_K3.npy')])

Train_Times = np.concatenate([np.load('Content_Times_Code_Training.npy'),
                              np.load('Content_Times_Code_Training.npy')])

Train_Direc = np.concatenate([np.load('../CWB/CWBEPA_Content_WIND_DIREC_code_Training_K3.npy'),
                             np.load('../CWB/CWBEPA_Content_WIND_DIREC_code_Training_K3.npy')])

Train_Speed = np.concatenate([np.load('../CWB/CWBEPA_Content_WIND_SPEED_code_Training_K3.npy'),
                             np.load('../CWB/CWBEPA_Content_WIND_SPEED_code_Training_K3.npy')])

Test_Rain = np.load('../CWB/CWBEPA_Content_RAIN_code_Testing_K3.npy')
Test_Rh = np.load('../CWB/CWBEPA_Content_RH_code_Testing_K3.npy')
Test_Temp = np.load('../CWB/CWBEPA_Content_TEMP_code_Testing_K3.npy')
Test_Times = np.load('Content_Times_Code_Testing.npy')
Test_Direc = np.load('../CWB/CWBEPA_Content_WIND_DIREC_code_Testing_K3.npy')
Test_Speed = np.load('../CWB/CWBEPA_Content_WIND_SPEED_code_Testing_K3.npy')

In [4]:
x = Input(shape=(348, 204, 1))
AIRSAT_Autoencoder_weight = glob_all('Unpair_Target_Embedding_AIRSAT_channel'+str(channel)+'_K3Area')[0]
AIRSAT_Encoder_model = Encoder(channel);AIRSAT_Decoder_model = Decoder(channel)
AIRSAT_Autoencoder = Model(x,AIRSAT_Decoder_model(AIRSAT_Encoder_model(x)))
AIRSAT_Autoencoder.load_weights(AIRSAT_Autoencoder_weight)
for layer in AIRSAT_Encoder_model.layers:layer.trainable = False
for layer in AIRSAT_Decoder_model.layers:layer.trainable = False
for layer in AIRSAT_Autoencoder.layers:layer.trainable = False

In [ ]:
K.clear_session()
image_shape = dataB[0].shape
g_model_AtoB = define_generator_content(image_shape,channel,resnet_n)
print('generator: A -> B')
g_model_BtoA = define_generator(image_shape,channel,resnet_n)
print('generator: B -> A')
d_model_A = define_discriminator(image_shape, d_lr)
print('discriminator: A -> [real/fake]')
d_model_B = define_discriminator_content(image_shape, d_lr)
print('discriminator: B -> [real/fake]')
c_model_AtoB = define_composite_model_A2B(g_model_AtoB, d_model_B, g_model_BtoA, image_shape, g_lr)
print('composite: A -> B -> [real/fake, A]')
c_model_BtoA = define_composite_model_B2A(g_model_BtoA, d_model_A, g_model_AtoB, image_shape, g_lr)
print('composite: B -> A -> [real/fake, B]')

train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA,
      AIRSAT_Encoder_model, AIRSAT_Decoder_model, dataA, dataB,
      Train_Rain,Train_Rh,Train_Temp,Train_Times,Train_Direc,Train_Speed,
      Test_Rain,Test_Rh,Test_Temp,Test_Times,Test_Direc,Test_Speed,
      model_path, EPA73_testing_PM25, station_coordinate, exArea_KNN, area_list,
      n_epochs=epochs, n_batch=batch, save_epochs = 1, channel = channel)

print('train models done')